# [Project] 신용카드 고객 신용 위험 예측

- 거래 데이터 분석을 통하여 고객 신용 위험을 예측하는 분류 모델 수행
- 신용 위험에 영향을 미치는 피처들에 대한 분석 수행

## 프로젝트 목적

프로젝트에서는 독일 금융 거래 고객 데이터를 바탕으로 금융 고객의 신용 위험을 예측해보는 분류 모델을 구현합니다. 데이터 분석 및 분류 모델을 바탕으로 새로운 고객의 데이터를 받았을 때, 신용 위험 여부를 예측할 수 있으며, 어떠한 특성 데이터가 위험 여부를 예측하는데 큰 영향을 미쳤는지 또한 알 수 있습니다.

## 가이드

1. *German Creditcard 데이터:** 데이터를 다운로드하고 컬럼 피처 구조와 내용을 확인
    

    1. 결측값: 비어 있는 데이터 또는 쓸모 없는 데이터를 삭제
    2. 데이터 시각화
        - 피처의 속성을 시각화
        - 피처와 Risk 의 관계를 분석    

    3. **데이터 전 처리
        - 문자 분류형을 수치 라벨과 원-핫 인코딩
        - 학습, 테스트 데이터 분리

2. **머신러닝 모델 학습:** 분류 모델을 사용하여 학습 수행<br>
    - 기본 분류 모델 학습 - 의사결정나무<br>
    - 다양한 분류 모델 학습<br>
    - 모델 튜닝 및 K-fold 교차 검증<br>

3. **평가 및 예측:** 학습된 모델을 바탕으로 평가 및 예측 수행<br>
    - Confusion Matrix<br>
    - 테스트 데이터의 예측값 출력<br>



In [ ]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras

print(tf.__version__)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns
%matplotlib inline

In [ ]:
# font_path = 'C:/Windows/Fonts/NanumGothic.ttf'
# font_path = '/Users/qkboo/Library/Fonts/NanumGothic.otf'
font_path = "/usr/share/fonts/truetype/nanum/NanumGothic.ttf"
fontname = fm.FontProperties(fname=font_path, size=16).get_name()  # 폰트 패밀리 이름!
plt.rc('font', family=fontname)  #  'NanumGothic'
# plt.rcParams["font.family"] = fontname

plt.rcParams['axes.unicode_minus'] = False #glypy 8722: Axes에 - 표시 안되는 것
plt.title('한글 타이틀...')

---

# 1. German Creditcard 데이터
-  https://archive.ics.uci.edu/ml/datasets/statlog+(german+credit+data)

### 데이터 다운로드
https://archive.ics.uci.edu/ml/datasets/statlog+(german+credit+data) 의 실제 데이터를 정리한 kaggle 의 `german_credit_data.csv`  파일를 사용한다.

 - https://www.kaggle.com/datasets/uciml/german-credit


> 

### 데이터 속성 확인

pandas를 사용하여 `german_credit_data.csv` 데이터를 읽고 속성을 파악해 본다.

- Age (Numeric: Age in years)
- Sex (Categories: male, female)
- Job (Categories : 0 - unskilled and non-resident, 1 - unskilled and resident, 2 - skilled, 3 - highly skilled)
- Housing (Categories: own, rent, or free)
- Saving accounts (Categories: little, moderate, quite rich, rich)
- Checking account (Categories: little, moderate, rich)
- Credit amount (Numeric: Amount of credit in DM - Deutsche Mark)
- Duration (Numeric: Duration for which the credit is given in months)
- Purpose (Categories: car, furniture/equipment, radio/TV, domestic appliances, repairs, education, business, vacation/others)
- Risk (0 - Person is not at risk, 1 - Person is at risk(defaulter))


In [ ]:
df = pd.read_csv("data/german_credit_data.csv")
df.head()

In [ ]:
df.columns

In [ ]:
df.info()

기술통계 표를 출력해 보자.

In [ ]:
df.describe()

## 결측값 확인

1. **결측값(missing value)** 또는 **이상치(outlier)**
1. 데이터 형식 변경 등

결측값을 확인한다.

In [ ]:
# 결측값 정보를 출력합니다.
df.isna().sum()

### 작업용 DataFrame 으로 복사한다.

In [ ]:
# 작업용으로 df_work 복사한다. 
df_work = df.copy()

Saving accounts 는 범주형으로 전체 값의 구성을 value_counts 로 확인하자

In [ ]:
df_work['Saving accounts'].value_counts()

'Saving accounts' ,'Checking account' 변수에 있는 결측값을 `Others`로 추가해 준다.

In [ ]:
df_work['Saving accounts'] = df_work['Saving accounts'].fillna('Others')
df_work['Checking account'] = df_work['Checking account'].fillna('Others')

In [ ]:
df_work.isna().sum()

In [ ]:
df_work['Saving accounts'].value_counts()

In [ ]:
df_work['Checking account'].value_counts()

In [ ]:
df_work.info()

## 데이터 시각화

각 변수 분포를 알아보기 위하여 시각화를 수행하겠습니다.

### - `Age` 중심 분석

In [ ]:
plt.figure(figsize=(15,5))

# sns.set(style='darkgrid')
ax = sns.countplot(x='Age', data=df_work)

### `- Sex` 기준 분석

Sex 속성의 요소별 개수를 확인해 본다.

In [ ]:
df_work['Sex'].value_counts()

seaborn 을 이용해 Sex 속성의 개수를 그려본다.

In [ ]:
sns.countplot(x='Sex', data=df_work)

value_counts() 결과를 plot으로 Sex 속성의 개수를 그려본다.

In [ ]:
df_work['Sex'].value_counts().plot(kind='bar')

Sex 속성 기준으로 신용위험 상태를 표시해 보자.

In [ ]:
sns.countplot(x='Risk', hue='Sex', data=df_work)

In [ ]:
sns.countplot(x='Sex', hue='Risk', data=df_work)

Sex 속성을 기준으로 나이 분포를 살펴보자.

In [ ]:
sns.displot(x='Age', hue='Sex', data=df_work)

### - `Job` 기준 분석

- Job (Categories : 0 - unskilled and non-resident, 1 - unskilled and resident, 2 - skilled, 3 - highly skilled)

In [ ]:
df_work['Job'].value_counts()

In [ ]:
sns.countplot(x='Job', data=df_work)

In [ ]:
df_work['Job'].value_counts().plot(kind='barh')

Job 속성과 신용위험을 비교해 보자

In [ ]:
sns.countplot(x='Job', hue='Risk', data=df_work)

### - `Housing ` 분석

In [ ]:
df_work['Housing'].value_counts()

In [ ]:
sns.countplot(x='Housing', data=df_work)

In [ ]:
df_work['Housing'].value_counts().plot(kind='barh')

Housing 속성과 신용위험을 비교해 보자

In [ ]:
sns.countplot(x='Housing', hue='Risk', data=df_work)

### -`Saving accounts ` 분석

Saving accounts 속성을 관련있는 속성과 비교한다.

- Saving accounts (Categories: little, moderate, quite rich, rich)


In [ ]:
df_work['Saving accounts'].value_counts()

In [ ]:
sns.countplot(x='Saving accounts', data=df_work)

In [ ]:
df_work['Saving accounts'].value_counts().plot(kind='barh')

Saving accounts 속성과 신용위험을 비교해 보자

In [ ]:
sns.countplot(x='Saving accounts', hue='Risk', data=df_work)

Sex 속성 기준해서 Saving accounts 속성을 비교하자

In [ ]:
sns.countplot(x='Saving accounts', hue='Sex', data=df_work)

### - `Checking account ` 시각화

Checking account 속성을 관련있는 속성과 비교한다.

- Checking account (Categories: little, moderate, rich)

In [ ]:
df_work['Checking account'].value_counts()

In [ ]:
sns.countplot(x='Checking account', data=df_work)

In [ ]:
df_work['Checking account'].value_counts().plot(kind='barh')

Checking account 속성과 신용위험을 비교해 보자

In [ ]:
sns.countplot(x='Checking account', hue='Risk', data=df_work)

Sex 속성 기준해서 Saving accounts 속성을 비교하자

In [ ]:
sns.countplot(x='Checking account', hue='Sex', data=df_work)

### - `Credit amount ` 분석

- Credit amount (Numeric: Amount of credit in DM - Deutsche Mark)

In [ ]:
# 수치형 데이터는 boxplot으로 분포를 출력합니다.
plt.boxplot(df_work['Credit amount'])

In [ ]:
sns.boxplot(x='Risk', y='Credit amount', data=df_work)

### - `Duration ` 시각화

In [ ]:
df_work['Duration'].head()

Duration 속성의 사분위수를 살펴보자
 - boxplot() 사용

In [ ]:
sns.boxplot(y='Duration', data=df_work)

In [ ]:
sns.boxplot(x='Risk', y='Duration', data=df_work)

Duration 속성을 Risk 기준에 맞춰 출력하자.

In [ ]:
sns.countplot(x='Duration', hue='Risk', data=df_work)
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.boxplot(df_work['Duration'])

In [ ]:
sns.boxplot(x='Risk', y='Duration', data=df_work)

### - `Purpose` 분석

 - Purpose (Categories: car, furniture/equipment, radio/TV, domestic appliances, repairs, education, business, vacation/others)

In [ ]:
df_work['Purpose'].value_counts()

Purpose 속성의 개수를 그린다.

In [ ]:
sns.countplot(x='Purpose', data=df_work)
plt.xticks(rotation=45)
plt.show()

In [ ]:
df_work['Purpose'].value_counts().plot(kind='barh')

Purpose 속성을 Risk 기준에 맞춰 출력하자.

In [ ]:
sns.countplot(x='Purpose', hue='Risk', data=df_work)
plt.xticks(rotation=45)
plt.show()

> 

### "ex)" 데이터에서  Age 속성 기준으로 전체와  `Risk` 속성 전부와 속성이 `bad`, `good` 인 구분해 밀집도 그래프를 그려보자.

 - seaborn.kdeplot() 사용

In [ ]:
sns.kdeplot(df_work['Age'], label='전연령')
sns.kdeplot(df_work[df_work['Risk']=='good']['Age'], label='신용good')
sns.kdeplot(df_work[df_work['Risk']=='bad']['Age'], label='신용bad')

plt.legend(title='Risk')
plt.show()

> 


### "ex)" 데이터에서 `Risk` 속성이 `bad`이고,  `Sex` 가 male 이고, `Saving accounts`가 모두 `little`인 샘플 수를 살펴보자

Saving accounts 에서 little 이 다수로 남녀별 비교를 해보자


 - `.loc[]` 연산자 사용

In [ ]:
test = df_work.loc[ (df_work['Risk'] == 'bad') & (df_work['Sex'] == 'male')  & (df_work['Saving accounts'] == 'little') ]
test.head()

위 결과를 이용해 Housing 기준으로 Duration 그래프를 그려보자.

In [ ]:
sns.countplot(x='Duration', hue='Housing', data=test)

## 데이터 전 처리

신용 위험은 Risk 속성에 대해서 good, bad 분류로 구분을 하기 때문에 여기서 분류 모델을 사용해서 모델을 구성한다.

데이터를 학습에 적합하게 준비하기 위해서 다음과 같은 전처리를 수행하겠습니다.

- Object 자료형 -> 숫자 자료형 변환하기
- 학습 데이터와 테스트 데이터로 나누기

### 수치형 변경

수치형 int64 크기를 int16 으로 줄여도 무방할 정도로 작은 범위 숫자를 사용한다.

In [ ]:
df_work.info()

Age, Job, Duration 속성의 최대 값을 확인해 보자
 - max() 사용

In [ ]:
df['Age'].max(), df['Job'].max(), df['Duration'].max()

Age, Job, Duration 속성에 int16으로 변경한다.
 - astype() 사용

In [ ]:
df['Age'] = df['Age'].astype(np.int16)
df['Job'] = df['Job'].astype(np.int16)
df['Duration'] = df['Duration'].astype(np.int16)

In [ ]:
df.info()

###  범주형 Object 를 수치 인코딩하자
 - LabelEncoder 사용
 -  get_dummies() 사용

In [ ]:
from sklearn.preprocessing import LabelEncoder

enclabel = LabelEncoder()

LabelEncoder 를 사용해서 Risk 속성을 수치 분류형으로 변환하고 이를 정답 레이블인 target 에 저장한다.

In [ ]:
target = enclabel.fit_transform( df_work['Risk'] )
target[:10]

Sex','Housing', 'Saving accounts', 'Checking account','Purpose' 속성은 분류형인 object 자료형인데 머신러닝 학습을 위해서 수치 분류형으로 변형해야 한`다.

get_dummies 함수를 사용해 범주 값에 원-핫 인코딩 적용해 보자.

In [ ]:
pd.get_dummies(df_work['Sex'], prefix='Sex')

In [ ]:
_features = df_work[ ['Sex','Housing', 'Saving accounts', 'Checking account','Purpose'] ]
_features

이제 주요 피처인 Sex','Housing', 'Saving accounts', 'Checking account','Purpose' 에 대해서 원-핫 인코딩을 해서 학습용 데이터로 저장하자.

In [ ]:
train = pd.get_dummies(_features)
train

In [ ]:
train = pd.concat([train], axis=1)
train.info()

In [ ]:
train.shape, target.shape

### 훈련/검증 데이터 분리

머신러닝의 성능을 평가 하기 위해서는 전체 데이터를 학습에 사용하지 않고 학습용 데이터와 테스트용 데이터를 나누어 사용합니다.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train, target, test_size = 0.2, random_state = 0)

In [ ]:
X_train.shape, X_test.shape

# 2. 머신러닝 모델 학습

분류 모델인 DecisionTree 를 사용하여 학습을 수행하고, 다양한 모델들을 살펴봅시다.

### 의사결정나무

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()

In [ ]:
model.fit(X_train, y_train)

print('Train score:', model.score(X_train, y_train))
print('Test score:', model.score(X_test, y_test))

### 다양한 분류 모델 학습

로지스틱 분류기 모델 이외의 다양한 분류 알고리즘을 사용하고 그 성능을 비교하여 봅시다.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb
from xgboost.sklearn import XGBClassifier


models = []
models.append(('LR', LogisticRegression(max_iter =5000))) 
models.append(('LDA', LinearDiscriminantAnalysis()))  # LDA 모델
models.append(('KNN', KNeighborsClassifier()))  # KNN 모델
models.append(('NB', GaussianNB())) 
models.append(('RF', RandomForestClassifier()))  # 랜덤포레스트 모델
models.append(('SVM', SVC(gamma='auto')))  # SVM 모델
models.append(('XGB', XGBClassifier()))  # XGB 모델

for name, model in models:
    model.fit(X_train, y_train)
    msg = "%s \t\t: Train score: %f \t\t Test score : %f" % (name, model.score(X_train, y_train), model.score(X_test, y_test))
    print(msg)

#### 앙상블인 XGBoost 에서 변수 중요도를 출력합니다.

In [ ]:
xgb.

In [ ]:
df_work.drop('Risk', axis=1)

In [ ]:
train.columns

In [ ]:
max_num_features = 20
ax = xgb.plot_importance(models[-1][1], height = 1, grid = True, importance_type = 'gain', show_values = False, max_num_features = max_num_features)

ytick = ax.get_yticklabels()
feature_importance = []
for i in range(max_num_features):
    # feature_importance.append(df_work.drop('Risk', axis=1).columns[int(ytick[i].get_text().split('f')[1])])
    feature_importance.append(train.columns[int(ytick[i].get_text().split('f')[1])])

ax.set_yticklabels(feature_importance)

plt.rcParams['figure.figsize'] = (10, 15)
plt.xlabel('The F-Score for each features')
plt.ylabel('Importances')
plt.show()

### 모델 튜닝 및 K-fold 교차 검증

머신러닝 모델들은 데이터의 특성에 잘 맞도록 다양한 파라미터를 조절하여 성능을 높일 수 있습니다. 이러한 과정을 모델 튜닝이라 하며 본 과정에서는 GridSearchCV를 사용하여 구현해보겠습니다. 추가로 k-fold 방식 또한 사용하여 학습 과정에서 생길 수 있는 과적합을 예방해 봅시다.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

# 성능 비교에 필요한 모델 파라미터들을 정의합니다.
parameters = { 'criterion': ['gini','entropy'], 
              # 'max_depth': np.arange(1,20,2),
              'min_impurity_decrease': np.arange(0, 1.0, 0.01) }
kfold = KFold(n_splits=5)

dt_clf = DecisionTreeClassifier()

gscv = GridSearchCV( dt_clf, parameters, scoring = 'accuracy', cv = kfold, n_jobs= -1)

In [ ]:
gscv.fit(X_train, y_train)

In [ ]:
# 결과를 출력합니다.
print(gscv.score(X_train, y_train))
print(gscv.score(X_test, y_test))

가장 성능이 좋았던 파라미터를 출력합니다.

In [ ]:
gscv.best_params_

장 성능이 좋았던 파라미터로 수행한 모델을 저장합니다.

In [ ]:
best_dt = gscv.best_estimator_
best_dt.score(X_test, y_test)

## 평가 및 예측

 - accuracy: 오차행렬로 양성/음성을 전체 수를 양성인 수의 비율로 나타낸다.
 - 재현율 Recall : 양성+음성 결과에 대해 양성의 비율
 
그러므로 신용 위험 예측에서 중요한 것은 위험 없음을 정확히 예측하는 것 보단 위험 있음 판단할 수 있다. 그래서 recall 방식은 `예측한 위험 있음` 대비 `실제 위험 있음`의 비율을 나타내기에 accuracy에서 놓칠 수 있는 결과 해석을 보충한다.

### Confusion Matrix

기존 score에서 볼 수 있었던 결과는 accuracy 기반의 결과였습니다. confusion matrix를 출력하여 각 class 별로 예측한 결과에 대해서 자세히 알아봅시다.

In [ ]:
from sklearn.metrics import confusion_matrix

model_predition = models[-1][1].predict(X_test)

cm = confusion_matrix(y_test, model_predition)

# 출력 파트 - seaborn의 heatmap을 사용
plt.rcParams['figure.figsize'] = (5, 5)
sns.set(style = 'dark', font_scale = 1.4)
ax = sns.heatmap(cm, annot=True)
plt.xlabel('Prediction')
plt.ylabel('Real Data')
plt.show()
cm

위 confusion matrix에서 x 축은 실제 데이터의 label을 의미하고 y 축은 예측한 데이터의 label을 의미합니다.

- **0,0 의 값:** `위험 없음(Pass)` 이라고 예측했을 때, 실제 데이터가 `위험 없음(Pass)`인 경우의 개수
- **0,1 의 값:** `위험 있음(Fail)` 이라고 예측했을 때, 실제 데이터가 `위험 없음(Pass)`인 경우의 개수
- **1,0 의 값:** `위험 없음(Pass)` 이라고 예측했을 때, 실제 데이터가 `위험 있음(Fail)`인 경우의 개수
- **1,1 의 값:** `위험 있음(Fail)` 이라고 에측했을 때, 실제 데이터가 `위험 있음(Fail)`인 경우의 개수

### 테스트 데이터의 예측값 출력

테스트 데이터를 하나씩 입력하여 그 결과를 출력해 봅시다.

In [ ]:
for i in range(10): 
    item = X_test.iloc[i].values
    prediction = models[-1][1].predict( item.reshape(1,-1))
    print("{} 번째 테스트 데이터의 예측 결과: {}, 실제 데이터: {}".format(i, prediction[0], y_test[i]))

>

---

# 참고

 - https://www.kaggle.com/datasets/uciml/german-credit
 - https://www.realcode4you.com/post/german-credit-analysis-using-python-data-science
 - `금융 거래 고객 신용 위험 예측`